In [4]:
%pip install psycopg2
import os
import scrapy
from scrapy.utils.project import get_project_settings
# from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner
from scrapy.loader import ItemLoader
from twisted.internet import reactor

%run E:\Ian\Ebay_Project\card_scraper\card_scraper\items.py

import re
# Reactor restart
from crochet import setup, wait_for
# %load_ext autoreload
# %autoreload 2

setup()



  Using cached psycopg2-2.9.3-cp39-cp39-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Ian\Virtual Environments\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# from webscrapingapi import WebScrapingApiClient

# client = WebScrapingApiClient(api_key='YOUR_API_KEY')

# response = client.get('https://webscrapingapi.com/', params ={
#     # API Parameters
#     # Set to 0 (off, default) or 1 (on) depending on whether or not to render JavaScript on the target web page. JavaScript rendering is done by using a browser.
#     'render_js': 1,
#     # Set datacenter (default) or residential depending on whether proxy type you want to use for your scraping request. Please note that a single residential proxy API request is counted as 25 API requests.
#     'proxy_type': 'datacenter',
#     # Specify the 2-letter code of the country you would like to use as a proxy geolocation for your scraping API request. Supported countries differ by proxy type, please refer to the Proxy Locations section for details.
#     'country': 'us',
#     # Set depending on whether or not to use the same proxy address to your request.
#     'session': 1,
#     # Specify the maximum timeout in milliseconds you would like to use for your scraping API request. In order to force a timeout, you can specify a number such as 1000. This will abort the request after 1000ms and return whatever HTML response was obtained until this point in time.
#     'timeout': 10000,
#     # Set desktop (default) or mobile or tablet, depending on whether the device type you want to your for your scraping request.
#     'device': 'desktop',
#     # Specify the option you would like to us as conditional for your scraping API request. Can only be used when the parameter render_js=1 is activated.
#     'wait_until': 'domcontentloaded',
#     # Some websites may use javascript frameworks that may require a few extra seconds to load their content. This parameters specifies the time in miliseconds to wait for the website. Recommended values are in the interval 5000-10000.
#     'wait_for': 0,
# }, headers={
#     # API Headers
#     'authorization': 'bearer test',
#     # Specify custom cookies to be passed to the request.
#     'cookie': 'test_cookie=abc; cookie_2=def'
# })

# print(response.status_code)
# print(response.headers);
# print(response.content);

""" 
import http.client

conn = http.client.HTTPSConnection("api.webscrapingapi.com")

conn.request("GET", "/v1?api_key=snejc2HGikGwZvrKMLJx5FTz42wOAmHZ&url=https%3A%2F%2Fapi.ipify.org%2F%3Fformat%3Djson&device=desktop&proxy_type=datacenter&session=500")

res = conn.getresponse()
data = res.read()

proxy = data.decode("utf-8") """


In [ ]:
""" print(proxy)
proxy = proxy.replace("{\"ip\":\"", "")
proxy = proxy.replace("\"}", "")
print(proxy)
proxy = "http://"+proxy
os.environ['http_proxy'] = proxy  """

{"ip":"2.56.103.151"}
2.56.103.151


In [ ]:



translate = {'青眼の白龍':'Blue-Eyes White Dragon',
             'ブルーアイズ・ホワイト・ドラゴン':'Blue-Eyes White Dragon',
             '遊★戯★王':'Yu-Gi-Oh!','ユーギオオー!':'Yu-Gi-Oh!',
             '遊戯王':'Yu-Gi-Oh!'}

class 青眼の白龍_SM_51_sold_spider(scrapy.Spider):
    name = 'BEWD'
    allowed_domains = ['auctions.yahoo.co.jp']
    start_urls =['https://auctions.yahoo.co.jp/closedsearch/closedsearch?va=%E9%9D%92%E7%9C%BC%E3%81%AE%E7%99%BD%E9%BE%8D&b=1&n=50&auccat=&tab_ex=commerce&ei=utf-8&aq=-1&oq=&sc_i=&exflg=&p=%E9%9D%92%E7%9C%BC%E3%81%AE%E7%99%BD%E9%BE%8D+sm-51&x=0&y=0','https://auctions.yahoo.co.jp/closedsearch/closedsearch?va=%E9%9D%92%E7%9C%BC%E3%81%AE%E7%99%BD%E9%BE%8D+lb&b=1&n=50&select=3&auccat=&tab_ex=commerce&ei=utf-8&aq=-1&oq=&sc_i=&exflg=&p=%E9%9D%92%E7%9C%BC%E3%81%AE%E7%99%BD%E9%BE%8D+lb-01&x=0&y=0']
    custom_settings = {'DOWNLOAD_DELAY':2}
    #scrape links of pages to scrape
    def parse(self,response):
        #location of item page link for all items on page
        all_links = response.xpath('//h3//a/@href').extract()
        for link in all_links:
            yield scrapy.Request(link,callback = self.item_parse)
        # Yields next page of search to scrape
        next_page = response.xpath('//ul[@class="Pager__lists"]//li//a[@data-ylk = "rsec:pagination;slk:next;pos:1"]/@href')
        yield scrapy.Request(next_page,self.parse)
    def item_parse(self,response):
        #Call Item Loader to parse the responses
        l = ItemLoader(item=CardScraperItem(), response=response)
        l.add_xpath('auction_id','//*[@id="l-main"]//div[@class = "l-right"]//li[6]//dd/text()')
        l.add_xpath('title','//*[@id="ProductTitle"]/div/h1//text()')    
        l.add_xpath('price','//*[@id="l-sub"]//dd[@class = "Price__value"]/text()')
        l.add_xpath('bids','//dd[@class = "Count__number"]/text()')
        l.add_xpath('tax','//*[@id="l-sub"]//dd[@class = "Price__value"]/span/text()')
        l.add_xpath('auction_start','//*[@id="l-main"]//div[@class = "l-left"]//li[2]//dd/text()')
        l.add_xpath('auction_end','//*[@id="l-main"]//div[@class = "l-left"]//li[3]//dd/text()')
        l.add_xpath('auction_extension','//*[@id="l-main"]//div[@class = "l-left"]//li[4]//dd/text()')
        l.add_xpath('best_offer_accepted','//*[@id="l-main"]//div[@class = "l-left"]//li[5]//dd/text()')
        l.add_xpath('image_list','//div[contains(@class,"ProductImage__body")]//img/@src')
        return l.load_item()
    
    
# process = CrawlerProcess(settings={'FEEDS':{'items.csv': {'format':'csv'}}})



# process = CrawlerProcess(get_project_settings())
# process.crawl(青眼の白龍_SM_51_sold_spider)
# process.start()
# !scrapy crawl BEWD



In [ ]:
from twisted.internet import reactor
from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from scrapy.utils.log import configure_logging

# Enable logging for CrawlerRunner

configure_logging()

runner = CrawlerRunner(settings=get_project_settings())

runner.crawl(青眼の白龍_SM_51_sold_spider)

runner.addBoth(lambda _: reactor.stop())

reactor.run() 


In [3]:
""" run_spider() """

NameError: name 'run_spider' is not defined

In [1]:
""" from multiprocessing import Process

from scrapy.crawler import CrawlerRunner
from scrapy.utils.project import get_project_settings
from scrapy.utils.log import configure_logging
# from apscheduler.schedulers.blocking import BlockingScheduler

# from myprojectscraper.spiders.my_homepage_spider import MyHomepageSpider
# from myprojectscraper.spiders.my_spider import MySpider

from twisted.internet import reactor

# Create Process around the CrawlerRunner
class CrawlerRunnerProcess(Process):
    def __init__(self, spider):
        Process.__init__(self)
        self.runner = CrawlerRunner(get_project_settings())
        self.spider = 青眼の白龍_SM_51_sold_spider

    def run(self):
        deferred = self.runner.crawl(self.spider)
        deferred.addBoth(lambda _: reactor.stop())
        reactor.run(installSignalHandlers=False)

# The wrapper to make it run multiple spiders, multiple times
def run_spider(spider):
    crawler = CrawlerRunnerProcess(spider)
    crawler.start()
    crawler.join()

# Enable logging when using CrawlerRunner
configure_logging()

run_spider(青眼の白龍_SM_51_sold_spider) """

NameError: name '青眼の白龍_SM_51_sold_spider' is not defined